<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/sitcom_T5_(with_nContext).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Big Bang Theory Exploratory Data Analysis

### Imports:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

!git clone https://github.com/adc257/info4940-sitcom.git

Cloning into 'info4940-sitcom'...
remote: Enumerating objects: 1082, done.
remote: Counting objects: 100% (1082/1082), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 1082 (delta 948), reused 1038 (delta 915), pack-reused 0
Receiving objects: 100% (1082/1082), 14.59 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (948/948), done.
Updating files: 100% (922/922), done.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
!python -m venv 3350
!source 3350/bin/activate

In [ ]:
# !pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install accelerate>=0.21.0
# !pip install --upgrade pip

In [ ]:
# pip uninstall transformers accelerate
# pip install transformers[torch]
# !pip install --upgrade setuptools

In [7]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics import f1_score, classification_report

In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration



In [3]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

In [4]:
def transformContextTarget(context, target):
  separator = "\n"
  quotation_token = '"'

  formatted_input = ""

  current_scene = context[0]['Scene']

  formatted_input += "Use the following as context:" + separator*2
  formatted_input += "The scene is: " + current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      formatted_input += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    formatted_input += separator


  formatted_input += separator + "-" *20 + separator*2

  formatted_input += "Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:" + separator*2


  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
  formatted_input += separator*2 + "Classify: [humorous] or [not humorous]."

  return formatted_input

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [5]:

# @title # Formatting the input
# @markdown ### Example raw entry:
print(data[3])
# @markdown ### Example formatted entry (3Context):



NameError: name 'data' is not defined

In [23]:
filePath = "/content/info4940-sitcom/cleaned-data/S1"
nSentenceContext = 3

episodePath = list_files(filePath)[0]

with open(episodePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print(f"Example raw entry:")
display(data[3])
print("\n\nExample formatted entry (3context):\n")
print(transformContextTarget(data[:3], data[3]))

print(f'Actual: {("isHumor" in data[3])}')

Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': 'Excuse me.',
 'Dialogue Start Time': '00:00:23:140000',
 'Dialogue End Time': '00:00:25:060000'}



Example formatted entry (3context):

Use the following as context:

The scene is: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." to Leonard.
Leonard says "Agreed. What's your point?" to Sheldon.
Sheldon says "There's no point, I just think it's a good idea for a T-shirt." to Leonard.

--------------------

Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:

The Scene is still: A corridor at a sperm bank.
Leonard says "Excuse me." to Sheldon and Receptionist.

Classify: [humorous] or [not humorous].
Actual: False


# Formatting the input

### Ex raw entry:

In [ ]:
# example entry
data[0]

NameError: name 'data' is not defined

In [ ]:
def generateDialogueWithContext(filePath, nSentenceContext):
  formatted_inputs = []
  labels = []

  for episodePath in list_files(filePath):

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]
        # formatted_entry = transformContextTarget(context, target)

        formatted_inputs.append(transformContextTarget(context, target))

        if "isHumor" in info:
          labels.append(1)
        else:
          labels.append(0)

        i += 1
  return formatted_inputs, labels

In [ ]:
dialogue_list1, label_list1 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S1', 3)
dialogue_list2, label_list2 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S2', 3)
dialogue_list3, label_list3 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S3', 3)
dialogue_list4, label_list4 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S4', 3)
dialogue_list5, label_list5 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S5', 3)

In [ ]:
print(dialogue_list2[3])
print("\n\n")
print("Actual humor label: ", label_list2[3])

Use the following as context:

The scene is: The stairwell.
Penny says "Yeah, but it doesn't really answer my question." to Leonard.
Leonard says "- What was your question again? " to Penny.
Leonard says "- Right. No. I'm lactose intolerant. " to Penny.

--------------------

Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:

The Scene is still: The stairwell.
Leonard says "- So, gas. " to Penny.

Classify: [humorous] or [not humorous].



Actual humor label:  0


In [ ]:
train_texts = dialogue_list1 + dialogue_list2 + dialogue_list3 + dialogue_list4
test_texts = dialogue_list5
train_labels = label_list1 + label_list2 + label_list3 + label_list4
test_labels = label_list5

In [ ]:
# Check if all the dialogues are strings:

# for i in range(len(test_texts)):
#   if type(test_texts[i]) != str:
#     print("NOT A STRING")
#     print("index:",i)
#     print("string:",test_texts[i])



# Change this Dialogue from a number to a string
test_texts[2106] = "1863.0"

### Baseline:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True,lowercase=True,stop_words='english')
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [ ]:
scores = []
predictions = []
clf = RandomForestClassifier(max_depth=5,n_jobs=-1).fit(X_train, train_labels)

predictions.append(clf.predict(X_test))
scores.append(clf.score(X_test, test_labels))

In [ ]:
print('Cross validated score: ', np.mean(scores))

Cross validated score:  0.5373617994662601


### Creating Text Embeddings:

In [ ]:
# Tokenize Data for BERT:
model_name = 'distilbert-base-cased'
device_name = 'cuda' # (you can use 'cpu' or 'mps')
max_length = 512

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


class MyDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
      self.encodings = encodings
      self.labels = labels

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      item['labels'] = torch.tensor(self.labels[idx])
      return item

  def __len__(self):
      return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
test_dataset = MyDataset(test_encodings, test_labels)


In [ ]:
# Loading pre-trained BERT Model:
model = DistilBertForSequenceClassification.from_pretrained(model_name,
                                                            num_labels=2).to(device_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
total_training_steps = len(train_dataloader) * num_train_epochs
warmup_proportion = 0.1

In [ ]:
training_args = TrainingArguments(
  num_train_epochs=3,              # total number of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=8,   # batch size for evaluation
  learning_rate=5e-5,              # initial learning rate for Adam optimizer
  warmup_steps= 1000,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
  weight_decay=0.01,               # strength of weight decay
  output_dir='./results',          # output directory
  logging_dir='./logs',            # directory for storing logs
  logging_steps= 0.2,               # number of steps to output logging (set lower because of small dataset size)
  evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [ ]:
# Fine tuning our BERT model:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  #score = accuracy_score(labels, preds)
  score = f1_score(labels, preds, average='weighted')
  return {
      'f1': score,
  }

In [ ]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset,
  compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,F1
768,0.651400,0.622280,0.658335
1536,0.617200,0.655059,0.601530
2304,0.586300,0.609760,0.665335
3072,0.450800,0.776657,0.655003


TrainOutput(global_step=3837, training_loss=0.5361927898407977, metrics={'train_runtime': 959.8334, 'train_samples_per_second': 63.933, 'train_steps_per_second': 3.998, 'total_flos': 2397379198627620.0, 'train_loss': 0.5361927898407977, 'epoch': 3.0})

In [ ]:
cached_model_directory_name = 'distill-bert-tuned-no-context'
trainer.save_model(cached_model_directory_name)

In [ ]:
# # Load the model above:
# current_directory = os.getcwd()
# model_directory = os.path.join(current_directory, cached_model_directory_name)

# saved_model_directory = "/path/to/your/directory/distill-bert-tuned-no-context"
# model = AutoModelForSequenceClassification.from_pretrained(model_directory)

In [ ]:
# Evaluating our Fine-Tuned Model:
trainer.evaluate()
predicted_results = trainer.predict(test_dataset)

predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list

print(classification_report(test_labels, predicted_labels))
print(classification_report(test_labels, predicted_labels, output_dict = True)['weighted avg']['f1-score'])

              precision    recall  f1-score   support

           0       0.64      0.79      0.71      2819
           1       0.67      0.49      0.57      2427

    accuracy                           0.65      5246
   macro avg       0.66      0.64      0.64      5246
weighted avg       0.66      0.65      0.64      5246

0.6445252731111133
